### Initialization

In [1]:
# import
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

In [2]:
# constants


In [3]:
# measured data


In [4]:
# theoretical functions


### Task 1

In [5]:
# calculations


In [6]:
# plots
